# DASH 

[Dash](https://plotly.com/dash/) is a web application framework that allows you to develop interactive dashboards in Python.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

With the [latest addition](https://github.com/plotly/jupyter-dash) `jupyter-dash`, we can render `dash` outputs `inline` 🎉🎊! Find about more in this [blogpost](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e).

In [3]:
# Let's import the brand new JupyterDash (05/20/2020)
from jupyter_dash import JupyterDash
# The following command uses Jupyter-Server-Proxy to authanticate Dash to communicate with JupyterHub. 
# Normally, Dash has nothing to do with Jupyter, but the following command magically binds them together :) 
JupyterDash.infer_jupyter_proxy_config()

## Example 1: A simple `textbox` 

### Define `app`
In a non-Jupyter environment, dash app is defined as following: 
```python
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)
```

In the notebook, we'll use `JupyterDash` instead of `dash.Dash`. The second argument `external_stylesheets` is to link an external `css`, giving our application its style.

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

### Define `server` (optional in Notebook)

When deploying our app to production, we will create a `server` (Flask) object for our `app`. The flask object implements a WSGI (Web Service Gateway Interface) application and acts as the central object. When we use this with `gunicorn`, a WSGI HTTP server, we can serve our Dash application to the public internet! 

Don't get intimidated at all, in the [next step of this brainhack school tutorial](https://github.com/agahkarakuzu/dash_heroku), we'll achieve this with a few clicks 😎 

In [ ]:
server = app.server

### Define the `layout`

The `layout` is the first main component of a `Dash` application. It decribes what the application looks like. 

The `layout` will be a tree of `html components` such as `div`,`h1`,`img`,`iframe` etc and `core components`, which are the interactive widgets. 
***
When we import:

```python
import dash_html_components as html
```

basic HTML components will be accesible from the `html` object such as `html.Div`,`html.H1`, `html.Img`,`html.Iframe` etc. Click [**here**](https://dash.plotly.com/dash-html-components) for the complete list of Dash HTML components.
***
When we import:

```python
import dash_core_components as dcc
```

many widgets will be accessible from the `dcc` object such as `dcc.CheckList`, `dcc.Input` (textbox), `dcc.Slider` etc. Click [**here**](https://dash.plotly.com/dash-core-components) for the complete list of Dash core components.
***

Let's define a layout that has:
* An input textbox 
* A simple `div`, a virtual box that can contain any type of HTML objects. 

In [ ]:
app.layout = html.Div([
    dcc.Input(id='my-textbox', value='🙄type sth', type='text'),
    html.Div(id='my-div')
])

Let's call our the `run_server` method of our `app`. It has the following options:

* `external (default)` It will print out application's URL, to open it in a new tab. 
* `inline` Renders application in the notebook
* `jupyterlab` Opens application in a new tab in Jupyter Lab.

`JupyterDash.run_server` method doesn't block execution of the notebook. It serves the app in a background thread, making it possible to run other notebook calculations while the app is running.

In [ ]:
#app.run_server(mode="inline")

## Define the `callback`

We decided which buttons to attach to our cockpit (`layout`). Let's define their function to take off! 🛫

**Dash uses data-driven callbacks via decorators.** It associates `Input` and `Output` objects through a `function` wrapped by the `@app.callback` decorator:

```python
@app.callback(Output(),[Input()])
def my_callback_function(input_var):
    return output_var
```

**It works automagically, let's understand what's happening here:**

These `Input` and `Output` objects are quite 🧠. Remember they come from the import:

```python
from dash.dependencies import Input, Output
```

Both objects require a `component_id` and a `component_property`:
* `Input`:
     * Take `component_property` from the layout entity with the `component_id` and assign it to `input_var`. 
***     
* `Output`:
     * Update `component_property` of the layout entity having `component_id` with `output_var` returned by `my_callback_function`.

Our first simple example will show the text entered in the textbox 🤯
 
⚠️**Execute callback cells once**, otherwise you will create clones of your callback output, which will throw an exception as `Output` won't know what to do with many of them :) 

In [ ]:
@app.callback(
    Output(component_id='my-div', component_property='children'),
    [Input(component_id='my-textbox', component_property='value')]
)
def update_output_div(input_value):
    return 'You\'ve entered "{}"'.format(input_value)

In [ ]:
# You can call run_server multiple times e.g., with different options.
app.run_server(mode="external")

### Fancy `layout` with `dash-bootstrap-components`

`dash-bootstrap-components` is a third party package that allows you to build consistently styled apps with complex, responsive layouts.

You can use dbc.themes as stylesheets with this component. Full list:

https://www.bootstrapcdn.com/bootswatch/

In [4]:
import dash_bootstrap_components as dbc

app = JupyterDash(__name__,external_stylesheets=[dbc.themes.DARKLY])

BHS_LOGO = "https://brainhackmtl.github.io/school2019/assets/images/logo_brainhack.png"

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(type="search", placeholder="Search")),
        dbc.Col(
            dbc.Button("Search", color="primary", className="ml-2"),
            width="auto",
        ),
    ],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# A navigation bar <3 Especially useful for multi-page dash apps (yes, it is possible!)
navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(html.Img(src=BHS_LOGO, height="40px")),
                    dbc.Col(dbc.NavbarBrand("School 2020", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://plot.ly",
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    color="dark",
    dark=True,
)

# A card! 
card = dbc.Card(
    [
        dbc.CardImg(src="https://i.ytimg.com/vi/PTYs_JFKsHI/hqdefault.jpg", top=True),
        dbc.CardBody(
            [
                html.H4("About", className="card-title"),
                html.P(
                    "Amazing people learning amazing stuff, "
                    "still fun despite COVID-19.",
                    className="card-text",
                ),
                dbc.Button("Modal window",id="open", color="primary"),
            ]
        ),
    ],
    style={"width": "18rem","margin-top":"1rem","margin-left":"1rem"},
)

# A modal window 
modal = dbc.Modal(
            [
                dbc.ModalHeader("No way!"),
                dbc.ModalBody("You can have modal windows 😻"),
                dbc.ModalFooter(
                    dbc.Button("Close", id="close", className="ml-auto")
                    
                ),
            ],
            id="modal",
        )






app.layout = html.Div([
    navbar,
    modal,
    dbc.Row([
        dbc.Col(card,width=4),
        dbc.Col(dbc.Progress(value=75, striped=True, animated=True,color="warning"),width={"size": 3, "offset": 1},align="center"),
        dbc.Col([
            dbc.Spinner(color="primary", type="grow"),
            dbc.Spinner(color="secondary"),
            dbc.Spinner(color="success", type="grow"),
            dbc.Spinner(color="warning"),
            dbc.Spinner(color="danger", type="grow"),
            dbc.Spinner(color="info"),
            dbc.Spinner(color="light", type="grow")
        ],width={"size": 4},align="center")
    ])
])

See how the `State` object is used to detect if the modal window is open.

In [5]:
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks")],
    [State("modal", "is_open")],
)
def toggle_modal(n1, n2, is_open):
    if n1 or n2:
        return not is_open
    return is_open

In [6]:
app.run_server('inline')

## Example 2: Update a `plot` with a `slider`

In the following example, the `value` property of the `Slider` is the input of the app and the output of the app is the `figure` property of the `Graph`. 

Whenever the `value` of the `Slider` changes, Dash evokes the callback function `update_figure` with the new `value`. The function filters the dataframe `df` with this new `value`, constructs a `figure` object, and returns it to the Dash application.

There are a few nice patterns in this example:

* We're using the Pandas library for importing and filtering datasets in memory. 

* We load our dataframe at the start of the app: `df = pd.read_csv('...')`. This dataframe df is in the global state of the app and can be read inside the callback functions.

* Loading data into memory can be expensive. By loading querying data at the start of the app instead of inside the callback functions, we ensure that this operation is only done when the app starts. When a user visits the app or interacts with the app, that data (the `df`) is already in memory. If possible, expensive initialization (like downloading or querying data) should be done in the global scope of the app instead of within the callback functions.

* The callback does not modify the original data, it just creates copies of the dataframe by filtered through pandas filters. This is important: your callbacks should never mutate variables outside of their scope. If your callbacks modify global state, then one user's session might affect the next user's session and when the app is deployed on multiple processes or threads, those modifications will not be shared across sessions.

In [ ]:
import pandas as pd
import plotly.graph_objs as go

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/'
    'datasets/master/gapminderDataFiveYear.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    [Input('year-slider', 'value')])
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]
    traces = []
    for i in filtered_df.continent.unique():
        df_by_continent = filtered_df[filtered_df['continent'] == i]
        traces.append(go.Scatter(
            x=df_by_continent['gdpPercap'],
            y=df_by_continent['lifeExp'],
            text=df_by_continent['country'],
            mode='markers',
            opacity=0.7,
            marker={
                'size': 15,
                'line': {'width': 0.5, 'color': 'white'}
            },
            name=i
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'type': 'log', 'title': 'GDP Per Capita'},
            yaxis={'title': 'Life Expectancy', 'range': [20, 90]},
            margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }


In [ ]:
app.run_server('inline')

## Example 3: Update a plot with multiple inputs

In Dash, any `Output` can have multiple `Input` components. 

Here's a simple example that binds five `Input`s:
* 2 `Dropdown`
* 2 `RadioItem`
* 1 `Slider`

to the `figure` property of the Graph component.

Notice how the `app.callback` lists all five `dash.dependencies.Input` inside a list in the second argument.

In [ ]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'cb5392c35661370d95f300086accea51/raw/'
    '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
    'indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value'),
     Input('year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }

app.run_server('inline')

## See more examples

See this [great collection of dash applications](https://github.com/plotly/dash-sample-apps/tree/master/apps).

P.S. There is a cool [🧠viewer](https://github.com/plotly/dash-sample-apps/tree/master/apps/dash-brain-viewer).


<center><h2> What's next?</h2></center>

It is obvious that we can easily run this notebook on Binder. We can even use `Voila` to make our dashboards like standalone web applications. 

Would not it be really cool to `iframe` our dashboard in a blogpost? To do that, we need to deploy our dash application to production. 

***

<center><h3> Dash + Heroku + Github Actions</h3></center>
